In [36]:
import requests
import simplejson
import unidecode
import operator
import json
import sys

In [37]:

##########################################################################################
# PRODUCTO A BUSCAR

product = "luz usb"
product_to_avoid = "xxx"

min_price_list = (1,501,1000,3000,8000,16000)
max_price_list = (500,1000,3000,8000,16000,240000)
# TUS BARRIOS SEPARADOS POR ESPACIO
other_locations = "ezpeleta"
# SI EL MAXIMO VENDIDO ES 1000, y % 0.7 SOLO MUESTRA LOS QUE VENDIERON ARRIBA DE 700 
percentage_max_sell = "0.05"
# USED o NEW
sell_condition = "new" #"old"
# MINIMUM SELL FOR LOCATION
min_sell = 1

file = open("mla_results_" + product + ".csv","a+")

#########################################################################################

In [42]:

def check_ALL_list_in_title(list,string ):
# ALL NEEDS TO MATCH
    result = 1
    for item in list:
        #print ("item:",item,"str:",string)
        if item.lower() in string.lower():
            rresult = 1
        else:
            result = 0
    return result


def check_ANY_list_in_title(list,string ):
# ANY NEEDS TO MATCH
    result = 0
    for item in list:
        #print ("item:",item,"str:",string)
        if item.lower() in string.lower() :
            result = 1
    return result



def search_mla_return_list(product,min_price,max_price):
    #print ("Returning complete MLA list for product",product)
    link = 'https://api.mercadolibre.com/sites/MLA/search?q=' + product 
    price_filter = "&price=" + str(min_price) + "-" + str(max_price)
    link2 = link + '&offset=' + str(0) + '&limit=' + str(50) + price_filter
    r = requests.get(link2)
    
    #print ("end")
    
    c = r.json()
    #print (c)
    c_orig = c
    c_partial = c['results']
    start =  c['paging']['offset']
    end =  c['paging']['limit']
    total =  c['paging']['total'] 
    total_saved = total
    total_c = c['results']
    maximum_sell = 0
    print ("Total(limited to 1000 with no token access):",total," start:",start," end ",end)
    if total > 1000:
        total = 1000
    num_new = 0
    num=0
    num_new = 0
    while ( total > 51 and end > 10 ): # and 0):  ## <--- full list instead of first 50
        start = start + end
        if ( (c['paging']['total'] - start) < end  ):
            end = c['paging']['total'] - 1 - start 
        link2 = link + '&offset=' + str(start) + '&limit=' + str(end) + price_filter
        print ("requesting LINK:",link2)
        r = requests.get(link2)
        c = r.json()
        c_partial = c['results']
        #print ("Total:",c['paging']['total']," start:",start," limit ",end)
        total_c = total_c + c_partial
        total = total - end 
        num = num + 1 
    return total_c

def return_max_sold_from_list(list,product,min_price,max_price):
    #print ("----- Searching max sold...");
    product_list = product.split()
    maximum_sell = 0
    num_new = 0
    for x in list[:]:
        #print(c['site_id'], c['results']['title'],c['results']['sold_quantity'], "$",c['results']['price'],c['site_id'], c['results']['address']['city_name'])
        sold_qty = list[num_new]['sold_quantity']
        price = list[num_new]['price']
        #print ("sold qty",sold_qty,"num:",num)
        title = unidecode.unidecode(list[num_new]['title']).lower()
        if ( sold_qty > maximum_sell and check_ALL_list_in_title(product_list,title ) and not check_ALL_list_in_title(product_to_avoid.split(),title ) and ( int(price) >= int(min_price) and int(price) <= int(max_price) )):
            maximum_sell = sold_qty
            #print ("new max sell found",maximum_sell)
        num_new = num_new + 1
    print ("....max units sold at this range: ",maximum_sell)
    return maximum_sell

def return_tuples_from_list(list,product,min_price,max_price):
    print ("----- Searching maximum units sold...")
    product_list = product.split()
    maximum_sell = 0
    num_new = 0
    for x in list[:]:
        #print(c['site_id'], c['results']['title'],c['results']['sold_quantity'], "$",c['results']['price'],c['site_id'], c['results']['address']['city_name'])
        sold_qty = list[num_new]['sold_quantity']
        price = list[num_new]['price']
        title = list[num_new]['title']
        title_list = title.split()
        title_short = title_list[0:3]
        #print ("Title short:",title_short)
        #print ("sold qty",sold_qty,"num:",num)
        title = unidecode.unidecode(list[num_new]['title']).lower()
        if ( sold_qty > maximum_sell and check_ALL_list_in_title(product_list,title ) and not check_ALL_list_in_title(product_to_avoid.split(),title ) and ( int(price) >= int(min_price) and int(price) <= int(max_price) )):
            maximum_sell = sold_qty
            #print ("Title short:",title_short) 
            #print ("new max sell found",maximum_sell)
        num_new = num_new + 1
    return maximum_sell


def print_products_from_list(total_c,product,condition,percentage_max_sell,max_sell,min_price,max_price):
    from operator import itemgetter
    print ("----- Printing products from list..max units sold:",max_sell," Percentage filter at:",percentage_max_sell,"...min:",min_price," max_price:",max_price)
    num = 0
    list_new = list()
    product_list = product.split()
    for c in total_c[:]:
        sold_qty = total_c[num]['sold_quantity']
        price = c['price']
        title = c['title']
        if ( (( sold_qty > ( max_sell * 0.1 ) or  sold_qty > 500 ) and total_c[num]['condition'].lower() == 'new' )  and ( check_ANY_list_in_title(product_list,title )) and not check_ALL_list_in_title(product_to_avoid.split(),title )  and ( int(price) >= int(min_price) and int(price) <= int(max_price) ) ):
            #print(unidecode.unidecode(c['title']),c['sold_quantity'], "$",c['price'],c['site_id'], unidecode.unidecode(c['address']['city_name']))
            line = unidecode.unidecode(c['title']) + "|" + str(c['sold_quantity']) + "|" + str(c['price']) + "|" + c['site_id'] + "|" + unidecode.unidecode(c['address']['city_name'])
            list_new.append(line.split("|"))
            line = line.replace(',',' ')
            line = line.replace('(',' ')
            line = line.replace(')',' ')
            line = line.replace('{',' ')
            line = line.replace('}',' ')
            line = line.replace('[',' ')
            line = line.replace(']',' ')
            line = line.replace('|',',')
            file.write(str(line) + '\n' )
        num = num + 1
    list_new = sorted(list_new, key=itemgetter(1), reverse=True)
    for l in list_new:
        print (l )

def print_products_from_list_in_locations(total_c,product,sell_condition,locations,min_sell,min_price,max_price):
    print ("----- Printing products from list in LOCATION..min $:",min_price," max $:",max_price)
    #sorted(total_c, key=lambda sold_quantity: sold_quantity[8])
    #sorted(total_c, key=operator.itemgetter(16) )
    product_list = product.split()
    other_location_list = locations.split()
    num = 0
    locations_list = locations.split()
    #print (json.dumps(total_c[num]))
    for c in total_c[:]:
        sold_qty = total_c[num]['sold_quantity']
        price = total_c[num]['price']
        title = unidecode.unidecode(c['title']).lower()
        location = unidecode.unidecode(c['address']['city_name'])
        if ( check_ANY_list_in_title(other_location_list,location )  and total_c[num]['condition'].lower() == sell_condition  and check_ALL_list_in_title(product_list,title ) and sold_qty >= min_sell and not check_ALL_list_in_title(product_to_avoid.split(),title ) and ( int(price) >= int(min_price) and int(price) <= int(max_price) )):
            print(title,c['sold_quantity'], "$",c['price'],c['site_id'], unidecode.unidecode(c['address']['city_name']))
        num = num + 1




In [ ]:
num = 0
for price in min_price_list:
    min_price = min_price_list[num]
    max_price = max_price_list[num]
    mla_list = search_mla_return_list(product,min_price,max_price)
    #return_tuples_from_list(mla_list,product,min_price,max_price)
    max_sold = return_max_sold_from_list(mla_list,product,min_price,max_price)

    ranger = '$' + str(min_price) + '-$' + str(max_price)
    print ("A#######BEGIN ANY LOCATION#######PRICE RANGE ",ranger," #######FOUND MAX UNITS SOLD ",max_sold,"#############")
    print_products_from_list(mla_list,product,sell_condition,percentage_max_sell,max_sold,min_price,max_price)
    print ("B#######BEGIN SPECIFIC LOCATION#######PRICE RANGE ",ranger," #######FOUND MAX UNITS SOLD ",max_sold,"#############")
    print_products_from_list_in_locations(mla_list,product,sell_condition,other_locations,0 , int(min_price), int(max_price))
    print ("########END#######################PRICE RANGE ",ranger," ############################################")
    num = num + 1

Total(limited to 1000): 1542  start: 0  end  50
requesting LINK: https://api.mercadolibre.com/sites/MLA/search?q=luz usb&offset=50&limit=50&price=1-500
requesting LINK: https://api.mercadolibre.com/sites/MLA/search?q=luz usb&offset=100&limit=50&price=1-500
requesting LINK: https://api.mercadolibre.com/sites/MLA/search?q=luz usb&offset=150&limit=50&price=1-500
requesting LINK: https://api.mercadolibre.com/sites/MLA/search?q=luz usb&offset=200&limit=50&price=1-500
requesting LINK: https://api.mercadolibre.com/sites/MLA/search?q=luz usb&offset=250&limit=50&price=1-500
requesting LINK: https://api.mercadolibre.com/sites/MLA/search?q=luz usb&offset=300&limit=50&price=1-500
requesting LINK: https://api.mercadolibre.com/sites/MLA/search?q=luz usb&offset=350&limit=50&price=1-500
requesting LINK: https://api.mercadolibre.com/sites/MLA/search?q=luz usb&offset=400&limit=50&price=1-500
requesting LINK: https://api.mercadolibre.com/sites/MLA/search?q=luz usb&offset=450&limit=50&price=1-500
requesti

requesting LINK: https://api.mercadolibre.com/sites/MLA/search?q=luz usb&offset=350&limit=50&price=501-1000
requesting LINK: https://api.mercadolibre.com/sites/MLA/search?q=luz usb&offset=400&limit=50&price=501-1000
requesting LINK: https://api.mercadolibre.com/sites/MLA/search?q=luz usb&offset=450&limit=50&price=501-1000
requesting LINK: https://api.mercadolibre.com/sites/MLA/search?q=luz usb&offset=500&limit=50&price=501-1000
requesting LINK: https://api.mercadolibre.com/sites/MLA/search?q=luz usb&offset=550&limit=50&price=501-1000
requesting LINK: https://api.mercadolibre.com/sites/MLA/search?q=luz usb&offset=600&limit=50&price=501-1000
requesting LINK: https://api.mercadolibre.com/sites/MLA/search?q=luz usb&offset=650&limit=50&price=501-1000
requesting LINK: https://api.mercadolibre.com/sites/MLA/search?q=luz usb&offset=700&limit=50&price=501-1000
requesting LINK: https://api.mercadolibre.com/sites/MLA/search?q=luz usb&offset=750&limit=50&price=501-1000
requesting LINK: https://api